**TASKS**:
- use the initialize() function to get the basis set instances and the jkfactory object 
- get overlap matrix (S) in the AO basis (basis_gen)
- set the U matrix -> block orthogonalize the overlap

Note that :

The density matrix in the AO basis and the corresponding matrix in the BO basis are related by
$$ D^{ao} = U \tilde{D} U^{\dagger} $$
Morover $ U^{-1} D^{ao} (U^{\dagger})^{-1} = \tilde{D} $. $U$ is not orthogonal ($U^{-1} != U^{\dagger}$).

Correspondingly the matrix representation of a given operator in the BO basis can be obtained from the AO basis representation as:
$$ \tilde{O} = U^{\dagger} O^{ao} U $$

In [1]:
### 
import os
import sys
import psi4
import numpy as np

sys.path.insert(0, "./common")
modpaths = os.environ.get('MODS_PATH')

if modpaths is not None :
    for path in modpaths.split(";"):
        sys.path.append(path)
from scf_run import run
from init_run import initialize
from Fock_helper import fock_factory
print("test mixed basis / functionals")
func_high = 'b3lyp'
func_low = 'blyp'
bset,bsetH, molelecule_str, psi4mol, wfn, jkobj = initialize(False,'DIRECT','3-21G','3-21G','H2O1.xyz',\
                   func_high,func_low,0,eri='nofit')

mints = psi4.core.MintsHelper(bset)
#I = np.array(mints.ao_eri())

H = np.array(mints.ao_kinetic())+ np.array(mints.ao_potential())
S = np.array(mints.ao_overlap())
numbas = bset.nbf() #get the number of basis function belonging to bset (the low-level-theory basis)

nbfA = bsetH.nbf() #get the number of basis function belonging to bsetH (the high-level-theory basis)

#make U matrix for blend basis(cc-pvdz+3-21G)
U = np.eye(numbas)
S11=S[:nbfA,:nbfA]
S11_inv=np.linalg.inv(S11)
S12 =S[:nbfA,nbfA:]
P=np.matmul(S11_inv,S12)
U[:nbfA,nbfA:]=-1.0*P

#S block orthogonal
Stilde= np.matmul(U.T,np.matmul(S,U))


mtest=np.zeros((nbfA,(numbas-nbfA)))
print("AB block of overlap (in BO basis) is zero: %s" %(np.allclose(Stilde[:nbfA,nbfA:],mtest)))


test mixed basis / functionals
High Level functional : b3lyp

Low Level functional : blyp

Low Level basis : 3-21G

High Level basis : 3-21G

0 1
O1   0.000000000000   -0.000014000000   -0.348240000000
H1   0.000000000000   0.760011000000   -0.932852000000
H                     0.000000000000    -0.759996000000    -0.932908000000
symmetry c1
no_reorient
no_com

  Memory set to   1.863 GiB by Python driver.
    Molecular point group: c1
    Full point group: Cs

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center              X                  Y                   Z               Mass       
    ------------   -----------------  -----------------  -----------------  -----------------
         O            0.000000000000    -0.000014000000    -0.348240000000    15.994914619570
         H            0.000000000000     0.760011000000    -0.932852000000     1.007825032230
         H            0.000000000000    -0.759996000000    -0.932908000000     1.007825032230

cent

---------------------------------------------------------
**Use fock_factory to get the Fock in BO basis**

A fock_factory instance can be initialized as:

fock_engine = **fock_factory**(jk_fact,Hmat,ovapm,*funcname*=None,*basisobj*=None,*exmodel*=0)

input: 

- jk_fact: the jk object obtained from the "initialize" function
- Hmat: the $ H_{core}$ matrix in the AO basis
- ovapm: the basis function overlap matrix. Only needed when we work in the density matrix framework, i.e
  forming $ SDS $, and diagonalizing the "density operator" we get natural orbitals. ovapm can be either $\tilde{S}$   or $S^{ao}$ depending on the basis we are working, respectively, BO or AO basis
- funcname: the functional corresponding to the low level theory, when BO embedding is intended. Otherwise is the     usual functional in a supermolecular calculation.
- basis object: same as funcname
- exmodel : exchange model, 0 is assumed, see Manby-Miller and Parkhill papers

fock_factory has many methods:

- **get_bblock_Fock**(self,*Cocc*=None,*Dmat*=None,*func_acc*=None,*basis_acc*=None,*U*=None,*return_ene*=False). This function can accept either the matrix containing MO coefficients or the density matrix (in the appropriate basis). **basis_acc** and **func_acc** denote the 'high-level-theory' basis object and functional respectively. $U$ has to be passed as numpy.ndarray 
- **get_Fock**(self,*Cocc*=None,*Dmat*=None,*return_ene*=False). This method can be used to get the usual supermolecular Fock matrix corresponding to $C_{occ}$/$D_{mat}$, *funcname* and *basisobj* (see the intialization)




**TASK**
- test the b3lyp-in-blyp case
- perform a full scf calculation using scf_run.run()
- run() functions declaration: 

**run**(jkclass,embmol,bset,bsetH,guess,func_h,func_l,exmodel,wfn)
- jkclass : True|False, enable native Psi4 JK class 
- embmol  : Psi4 molecule object
- bset    : total basis set object
- bsetH   : 'high-level-theory' fragment basis set object
- guess   : specify the type of initial guess 'GS'|'SAD' (i.e ground state/sum of atomic densities)
- func_h  : 'high-level-theory' fragment functional
- func_l  : 'low-level-theory' fragment functional
- exmodel : exact exchange model, default = 0
- wfn     : Psi4 wavefunction object (correspondig to 'func_l' run

In [2]:
Cocc = np.array(wfn.Ca_subset('AO','OCC'))

try:
    U_inv = np.linalg.inv(U)
except np.linalg.LinAlgError:
    print("Error in numpy.linalg.inv of inputted matrix")

Cocc = np.matmul(U_inv,Cocc)
Dinput = np.matmul(Cocc,Cocc.T)
fockbase = fock_factory(jkobj,H,Stilde,funcname=func_low,basisobj=bset) #Stilde as overlap matrix
F_bblock = fockbase.get_bblock_Fock(Dmat=Dinput,func_acc=func_high,basis_acc=bsetH,U=U)

print("F(BO) dim: %i,%i\n" % (F_bblock.shape[0],F_bblock.shape[1]))



F(BO) dim: 13,13



In [3]:

res, wfnBO = run(jkobj,psi4mol,bset,bsetH,'GS',func_high,func_low,0,wfn)


Overlap_AA in BO is Overlap_AA in AO: True
Overlap_AB in BO is zero: True
using EX model: ......... 0


Number of occupied orbitals: 5

using as guess the density from the low level theory hamiltonian

Start SCF iterations:


SCF Iteration   1: Energy = -75.9665485640846612   dE = -7.59665E+01   dRMS = 2.94704E-03
SCF Iteration   2: Energy = -75.9671084826183431   dE = -5.59919E-04   dRMS = 7.81185E-04
SCF Iteration   3: Energy = -75.9671176372775818   dE = -9.15466E-06   dRMS = 8.30754E-04
SCF Iteration   4: Energy = -75.9671555308781308   dE = -3.78936E-05   dRMS = 3.54256E-05
SCF Iteration   5: Energy = -75.9671556611094587   dE = -1.30231E-07   dRMS = 5.26206E-06
SCF Iteration   6: Energy = -75.9671556632927576   dE = -2.18330E-09   dRMS = 2.26854E-07
SCF Iteration   7: Energy = -75.9671556632983282   dE = -5.57066E-12   dRMS = 3.42207E-08
SCF Iteration   8: Energy = -75.9671556632982998   dE =  2.84217E-14   dRMS = 1.58172E-09
Total time for SCF iterations: 3.035 seconds 


Final 

In [4]:
#compare the b3lyp-in-blyp orbital enegies/total energy to pure blylp
eps_a = np.array( wfn.epsilon_a() )
ndocc = wfnBO['ndocc']
print('Doubly Occupied: (%s / %s-in-%s)\n' % (func_low,func_high,func_low))
for k in range(ndocc):
    print('%iA : %.6f | %.6f' %(k+1,eps_a[k],wfnBO['epsilon_a'][k]))

print('Virtual:\n')

for k in range(ndocc,numbas):
    print('%iA : %.6f | %.6f'% (k+1,eps_a[k],wfnBO['epsilon_a'][k]))


Doubly Occupied: (blyp / b3lyp-in-blyp)

1A : -18.637661 | -18.991038
2A : -0.890960 | -0.986969
3A : -0.446978 | -0.499914
4A : -0.255321 | -0.320740
5A : -0.185924 | -0.255990
Virtual:

6A : 0.071694 | 0.095875
7A : 0.162360 | 0.188004
8A : 0.879981 | 0.897964
9A : 0.952293 | 0.992618
10A : 1.370386 | 1.441790
11A : 1.447223 | 1.516456
12A : 1.571051 | 1.638742
13A : 2.679564 | 2.744814


In [5]:
#get Density matrix and Fock(0) from wfnBO

tD = wfnBO['Dmtx']
tFock = wfnBO['Fock']
U = wfnBO['Umat']
C = wfnBO['Ccoeff'] 
Stilde = wfnBO['Ovap'] 
# get rt params
from rt_mod import set_params
pulse_opts, calc_params = set_params('input.inp')
delta_t = calc_params['delta_t']


In [6]:
#initialize the mints object
mints = psi4.core.MintsHelper(bset)
#intialize fock_factory
from Fock_helper import fock_factory
Hcore = np.asarray(mints.ao_potential()) + np.asarray(mints.ao_kinetic())

fock_base = fock_factory(jkobj,Hcore,Stilde, \
                            funcname=func_low,basisobj=bset,exmodel=0)


In [7]:
#dip_mat is transformed in the BO basis
dipole=mints.ao_dipole()
direction = 2 # i.e z-dir
dip_mat=np.matmul(U.T,np.matmul(np.array(dipole[direction]),U))


class **real_time**():

initialization params:

- *Dinit* : intial density matrix (t=0)
- *Fock_init* : guess Fock matrix
- *fock_factory* : fock factory object
- *ndocc* : number of doubly occupied MO
- *basis* : supermolecular basis set object
- *Smat* : overlap matrix (representation dependent, can be either BO or AO)
- *pulse_opts* : pulse paramter dictionary
- *delta_t* : time-step
- *Cmat* : matrix used get the relevant quantities to the propagation basis Cmat = $C_{coeff}$ {or $S^{-1/2}$}
- *dipmat* : matrix representation of dipole operator (either on BO or AO basis)
- out_file=sys.stderr :output err file

the following parameters have defaults values, if not speciefied  the real-time object 
should lead to the same results of supermolecular calculation.

- basis_acc = None : the small 'high-level-theory' basis set object
- func_acc=None : the 'high-level-theory' functional
- U=None : the U matrix -> $\tilde{S} = U^{T}SU$
- local_basis=False : Bool, enable local basis (RLMO localization scheme)
- exA_only=False : Bool , restrict the excitation to basis_acc subblock elements (nbfA $\times$ nbfA) of dipole
- occlist=None : a list containting identifiers of occupied MOs, to be used in occ->virt excitation
- virtlist=None : a list containting identifiers of virtual MOs


In [8]:
from rt_base import real_time
rt_prop = real_time(tD, tFock, fock_base, ndocc, bset, Stilde, pulse_opts, delta_t, C, dip_mat,\
            out_file=sys.stderr,  basis_acc = bsetH, func_acc=func_high,U=U,local_basis=False,\
                                             exA_only=False,occlist=None, virtlist=None)

In [9]:
Dp_init , Da = rt_prop.init_boost()

Perturb density with analytic delta
Dipole matrix is transformed to the MO basis

Local basis: False



In [10]:
rt_prop()

converged after 1 interpolations
i is: 0
norm is: 0.000000171922
Trace(D)(t+dt) : 5.00000000


In [11]:
print(rt_prop.iter_num())

1


In [13]:
for k in range(rt_prop.iter_num(),10):
    rt_prop()

converged after 1 interpolations
i is: 1
norm is: 0.000000042118
Trace(D)(t+dt) : 5.00000000
converged after 1 interpolations
i is: 2
norm is: 0.000000047573
Trace(D)(t+dt) : 5.00000000
converged after 1 interpolations
i is: 3
norm is: 0.000000050089
Trace(D)(t+dt) : 5.00000000
converged after 1 interpolations
i is: 4
norm is: 0.000000041563
Trace(D)(t+dt) : 5.00000000
converged after 1 interpolations
i is: 5
norm is: 0.000000047978
Trace(D)(t+dt) : 5.00000000
converged after 1 interpolations
i is: 6
norm is: 0.000000047102
Trace(D)(t+dt) : 5.00000000
converged after 1 interpolations
i is: 7
norm is: 0.000000040438
Trace(D)(t+dt) : 5.00000000
converged after 1 interpolations
i is: 8
norm is: 0.000000047894
Trace(D)(t+dt) : 5.00000000
converged after 1 interpolations
i is: 9
norm is: 0.000000043508
Trace(D)(t+dt) : 5.00000000
